In [1]:
import cv2 
import os

In [161]:
# Load Images
Directories = ['Set5', 'Set14', 'Urban100', 'BSD100', 'Celeba_HQ']
scales = [2,4,8]
import re

def load_images_from_folder(folder):
    images = []
    os_list_range = len(os.listdir(folder))
    for filename in range(os_list_range):
        img = Image.open(folder + "/" + str(filename) + '.png')
        if img is not None:
            images.append(img)
    return images

In [69]:
# Downsample
from PIL import Image 
import PIL 

def downsample(): 
    for directory in Directories:
        images = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + '/HR/Original')
        for scale in scales: 
            disk_cropped = '/Users/ege/Desktop/DataSets/' + directory + '/HR/Original_Cropped/'
            disk_dir = '/Users/ege/Desktop/DataSets/' + directory + "/LR/" + str(scale) + "_b/"
            print(scale)
            for i, oriimg in enumerate(images): 
                width, height = oriimg.size
                
                Mod_1 = oriimg.size[0]%8
                Mod_2 = oriimg.size[1]%8
                right = width
                bottom = height

                cropped_img_1 = oriimg.crop((0, 0, right-Mod_1, bottom-Mod_2))
                
                width, height = cropped_img_1.size
                
                img = cropped_img_1.resize((int(width/scale),int(height/scale)), Image.BICUBIC)
                              
                cropped_img_1.save(disk_cropped +str(i)+ ".png","PNG")
                
                img.save(disk_dir +str(i)+ ".png","PNG")

In [3]:
from sewar.full_ref import mse, rmse, psnr, rmse_sw, uqi, ssim, ergas, scc, rase, sam, msssim, vifp, psnrb 

def evaluate(img1,img2):
    print(ssim(img1,img2))
    

In [151]:
# Upsample
# Load Images 

Directories = ['Set5', 'Set14', 'Urban100', 'BSD100', 'Celeba_HQ']
scales = [2,4,8]

def upsample():
    for directory in Directories:
        for scale in scales: 
            images = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + "/LR/" + str(scale) + '_b')
            disk_dir = '/Users/ege/Desktop/DataSets/' + directory + "/HR/" + str(scale) + "_upscaled/"
            for i, oriimg in enumerate(images): 
                width, height = oriimg.size
                img = oriimg.resize((int(width*scale),int(height*scale)), Image.BICUBIC)
                #img = cv2.resize(oriimg,None, fx=scale, fy=scale, interpolation = cv2.INTER_CUBIC)
                img.save(disk_dir +str(i)+ ".png","PNG")
        

In [11]:
import numpy 
import math
import cv2

def psnr_numpy(img1, img2):
    mse = numpy.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [170]:
# Evaluation using library and metric using rgb 

Directories = ['Set14']
scales = [4]

import numpy as np
import pandas as pd

from sewar.full_ref import mse, rmse, rmse_sw, uqi, ssim, ergas, scc, rase, sam, msssim, vifp, psnrb, psnr

metric_name = []
upsampling_method = []
downsampling_method = []
image_set = []
image_scale = [] 
image_number = []
metric_1 = []
metric_2 = []
metric_3 = []
metric_4 = []
metric_5 = []
metric_6 = []
metric_7 = []
metric_8 = []

for directory in Directories:    
    for scale in scales: 
        images_1 = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + "/HR/" + "Original_Cropped") 
        images_2 =  load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + "/HR/" + str(scale) + "_upscaled/")

        #images = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + '/HR')
        for i, oriimg in enumerate(images_1): 
            #evaluated_value = ssim(oriimg,images_2[i]) #evaluate(oriimg,images_2[i])
            #value_mse = uqi(images_1[i],images_2[i])
            #print(value_mse)
            
            pil_image_1 = images_1[i].convert('RGB') 
            pil_image_2 = images_2[i].convert('RGB')
            
            open_cv_image_1 = np.array(pil_image_1) 
            open_cv_image_2 = np.array(pil_image_2) 
            
            # Convert RGB to BGR 
            open_cv_image_1 = open_cv_image_1[:, :, ::-1].copy()
            open_cv_image_2 = open_cv_image_2[:, :, ::-1].copy()
            
            try:
                mse_value = mse(open_cv_image_1,open_cv_image_2) #evaluate(oriimg,images_2[i])
                rmse_value = rmse(open_cv_image_1,open_cv_image_2)
                psnr_value = psnr(open_cv_image_1,open_cv_image_2)
                rmse_sw_value = psnr_numpy(open_cv_image_1,open_cv_image_2)
                ssim_value = ssim(open_cv_image_1,open_cv_image_2)
                sam_value = sam(open_cv_image_1,open_cv_image_2)
                msssim_value = msssim(open_cv_image_1,open_cv_image_2)
                psnrb_value = psnrb(open_cv_image_1,open_cv_image_2) 
                
            except:
                mse_value = None
                rmse_value = None
                psnr_value = None
                ssim_value = None
                sam_value = None
                msssim_value = None
                psnrb_value = None
                
                print('Folder:' + str(scale) + '_upscaled image number:' + str(i))
                pass
            

            # Merge all the lists/Pandas Dataframes in one and then concatenate them as a LIST
            upsampling_method.append('bicubic')
            downsampling_method.append('bicubic')
            image_set.append(str(directory))
            image_scale.append(str(scale))
            image_number.append(str(i))
            metric_1.append(mse_value)
            metric_2.append(rmse_value)
            metric_3.append(psnr_value)
            metric_5.append(psnrb_value)
            metric_6.append(sam_value)
            metric_7.append(msssim_value)
            metric_8.append(sam_value)
            

            
d_1 =  pd.DataFrame(metric_1)
d_2 =  pd.DataFrame(metric_2)
d_3 =  pd.DataFrame(metric_3)
d_5 =  pd.DataFrame(metric_5)
d_6 =  pd.DataFrame(metric_6)
d_7 =  pd.DataFrame(metric_7)
d_8 =  pd.DataFrame(metric_8)
m_1 =  pd.DataFrame(upsampling_method)
m_2 =  pd.DataFrame(downsampling_method)
i_s =  pd.DataFrame(image_set)
i_m =  pd.DataFrame(image_scale)
i_n =  pd.DataFrame(image_number)


        
frames = [m_1,m_2,i_s,i_m,i_n,d_1,d_2,d_3,d_5,d_6,d_7,d_8]
result = pd.concat(frames, axis=1)

            # Save the big LIST again as Dataframe and then save it as list_2.csv
df = pd.DataFrame(result)
print(df)
df.to_csv('first_evaluation_test_2.csv', mode='a', header=False)

          0        0      0  0   0           0          0          0  \
0   bicubic  bicubic  Set14  4   0  340.186376  18.444142  22.813634   
1   bicubic  bicubic  Set14  4   1  587.927949  24.247226  20.437563   
2   bicubic  bicubic  Set14  4   2  613.605899  24.771070  20.251908   
3   bicubic  bicubic  Set14  4   3  285.003467  16.882046  23.582302   
4   bicubic  bicubic  Set14  4   4  235.425629  15.343586  24.412266   
5   bicubic  bicubic  Set14  4   5  154.345134  12.423572  26.245874   
6   bicubic  bicubic  Set14  4   6  321.561325  17.932131  23.058166   
7   bicubic  bicubic  Set14  4   7  259.378577  16.105234  23.991463   
8   bicubic  bicubic  Set14  4   8   95.067094   9.750236  28.350501   
9   bicubic  bicubic  Set14  4   9  166.562507  12.905910  25.915031   
10  bicubic  bicubic  Set14  4  10   83.656570   9.146397  28.905803   
11  bicubic  bicubic  Set14  4  11  610.575448  24.709825  20.273410   
12  bicubic  bicubic  Set14  4  12  270.753818  16.454599  23.80

In [171]:
print(d_3.mean(axis = 0))

0    24.230263
dtype: float64


In [57]:
def lookifaxisaresame(img1,img2):
    
    crp_img2 = img2.crop((0, 0, img1.size[0], img1.size[1]))

    return img1, crp_img2
    
    


In [165]:
#Evaluating by hardcoded psnr and sssim and metrics YCBCR information
Directories = ['Set14','BSD100','Set5','Urban100','Celeba_HQ']
scales = [2]

import numpy as np
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio, mean_squared_error
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2ycbcr
from skimage import data, img_as_float


from sewar.full_ref import mse, rmse, rmse_sw, uqi, ergas, scc, rase, sam, msssim, vifp, psnrb, psnr

Image_DataSet_List = []
Image_Scaled_List = []
Average_Metric_0_List = []
Average_Metric_1_List = []
Scale_List = []


for directory in Directories:  
    metric_name = []
    upsampling_method = []
    downsampling_method = []
    image_set = []
    image_scale = [] 
    image_number = []
    metric_1 = []
    metric_2 = []
    for scale in scales: 
        images_1 = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + "/HR/" + "Original_Cropped") 
        images_2 =  load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + "/HR/" + str(scale) + "_upscaled/")
        Scale_List.append(scale)   
        #images = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + '/HR')
        for i, oriimg in enumerate(images_1): 
            
            
            if (images_1[i].size != images_2[i].size):
                images_1[i], images_2[i] = lookifaxisaresame(images_1[i],images_2[i])
            
            pil_image_1 = images_1[i].convert('RGB') 
            pil_image_2 = images_2[i].convert('RGB')
            
            open_cv_image_1 = np.array(pil_image_1) 
            open_cv_image_2 = np.array(pil_image_2) 
            
            

            
            # Convert RGB to BGR 
            open_cv_image_1 = open_cv_image_1[:, :, ::-1].copy()
            open_cv_image_2 = open_cv_image_2[:, :, ::-1].copy()
            
            im1_t = np.atleast_3d(img_as_float(open_cv_image_1))
            im2_t = np.atleast_3d(img_as_float(open_cv_image_2))

            if im1_t.shape[2] == 1 or im2_t.shape[2] == 1:
                im1_t = im1_t[..., 0]
                im2_t = im2_t[..., 0]

            else:
                im1_t = rgb2ycbcr(im1_t)[:, :, 0:1] / 255.0
                im2_t = rgb2ycbcr(im2_t)[:, :, 0:1] / 255.0

            try: 
                psnr_val = peak_signal_noise_ratio(im1_t, im2_t)
                structural_similarity_val = ssim(im1_t,im2_t,win_size=11,gaussian_weights=True,multichannel=True,data_range=1.0,K1=0.01,K2=0.03,sigma=1.5)                #print(structural_similarity(open_cv_image_1, open_cv_image_2))


            except:
                print('error at: ' + str(i))
                psnr_val = None
                structural_similarity_val = None
                
                


            # Merge all the lists/Pandas Dataframes in one and then concatenate them as a LIST
            upsampling_method.append('bicubic')
            downsampling_method.append('bicubic')
            image_set.append(str(directory))
            image_scale.append(str(scale))
            image_number.append(str(i))
            metric_1.append(psnr_val)
            
            metric_2.append(structural_similarity_val)
          
        
        
    d_1 =  pd.DataFrame(metric_1)
    d_2 =  pd.DataFrame(metric_2)
    m_1 =  pd.DataFrame(upsampling_method)
    m_2 =  pd.DataFrame(downsampling_method)
    i_s =  pd.DataFrame(image_set)
    i_m =  pd.DataFrame(image_scale)
    i_n =  pd.DataFrame(image_number)

    frames = [m_1,m_2,i_s,i_m,i_n,d_1,d_2]
    result = pd.concat(frames, axis=1)
    df = pd.DataFrame(result)
    
    print(df)
    
    Image_DataSet_List.append(directory)
    Average_Metric_0_List.append(d_1.mean())
    Average_Metric_1_List.append(d_2.mean())
   
    

I_D_L =  pd.DataFrame(Image_DataSet_List)
A_M_0 =  pd.DataFrame(Average_Metric_0_List)
A_M_1 =  pd.DataFrame(Average_Metric_1_List)
S_L   =  pd.DataFrame(Scale_List)
    
frames_overall = [I_D_L,A_M_0,A_M_1,S_L]
result_overall = pd.concat(frames_overall, axis=1)
df_overall= pd.DataFrame(result_overall)

print(df_overall)


          0        0      0  0   0          0         0
0   bicubic  bicubic  Set14  2   0  30.666347  0.909632
1   bicubic  bicubic  Set14  2   1  26.585917  0.946814
2   bicubic  bicubic  Set14  2   2  26.005674  0.852417
3   bicubic  bicubic  Set14  2   3  27.942775  0.840608
4   bicubic  bicubic  Set14  2   4  29.265910  0.845599
5   bicubic  bicubic  Set14  2   5  32.931336  0.958816
6   bicubic  bicubic  Set14  2   6  27.815216  0.803186
7   bicubic  bicubic  Set14  2   7  29.133704  0.789878
8   bicubic  bicubic  Set14  2   8  34.145069  0.895031
9   bicubic  bicubic  Set14  2   9  32.717302  0.950129
10  bicubic  bicubic  Set14  2  10  35.004025  0.863210
11  bicubic  bicubic  Set14  2  11  24.530176  0.696286
12  bicubic  bicubic  Set14  2  12  30.523492  0.899284
13  bicubic  bicubic  Set14  2  13  33.257687  0.912279
          0        0       0  0   0          0         0
0   bicubic  bicubic  BSD100  2   0  32.314055  0.899080
1   bicubic  bicubic  BSD100  2   1  29.796060